# 作業 3



## 環境

請填寫當前執行使用的環境。

|環境|名稱|版本|
|-|-|-|
|作業系統|Windows|10|
|程式執行環境|jupyter notebook|1.0.0|
|python 版本|python3|3.6.9|
|安裝環境|pip|20.0.2|

## 安裝

請填寫安裝套件需要的指令

pip install -r requirements.txt


## 參考連結

- 有無參考他人之實驗結果：有
- 參考連結：

In [1]:
# 請勿更動此區塊程式碼

import time
import numpy as np
import pandas as pd

EXECUTION_START_TIME = time.time() # 計算執行時間
df = pd.read_csv('train.csv')    


## 資料分析與前處理


In [7]:
# 資料分析與前處理
import seaborn as sns   
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer       # 匯入填補缺失值的工具
from sklearn.preprocessing import LabelEncoder # 匯入 Label Encoder
from matplotlib import pyplot

std = df.describe().loc['std','Volume']
mean = df.describe().loc['mean','Volume']
df.Volume = df.Volume.apply(lambda x : (x - mean) / std)
df['Move'] = df['Close Price'].apply(lambda x : 'R')

for index, row in df.iterrows():
    
    if index == 0:
        df.at[index,'Move'] = '1'
    else:
        if df.at[index,'Close Price'] - df.at[index - 1,'Close Price']>=0:
            df.at[index,'Move'] = '1'
        else:
            df.at[index,'Move'] = '0'
            
train_x = df[['Open Price']]
train_y = df[['Move']]

df.head(20)

,Date,Open Price,Close Price,High Price,Low Price,Volume,Movement,Move
0,02-Jan-2009,902.99,931.80,934.73,899.35,0.813446,1,1
1,05-Jan-2009,929.17,927.45,936.63,919.53,1.823778,0,0
2,06-Jan-2009,931.17,934.70,943.85,927.28,1.808027,1,1
3,07-Jan-2009,927.45,906.65,927.45,902.37,1.299265,0,0
4,08-Jan-2009,905.73,909.73,910.00,896.81,1.511306,1,1
5,09-Jan-2009,909.91,890.35,911.93,888.31,1.307818,0,0
6,12-Jan-2009,890.40,870.26,890.40,864.32,1.314143,0,0
7,13-Jan-2009,869.79,871.79,877.02,862.02,1.530482,1,1
8,14-Jan-2009,867.28,842.62,867.28,836.93,1.819317,0,0
9,15-Jan-2009,841.99,843.74,851.59,817.04,3.594499,1,1


## Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder    # 匯入 Label Encoder
from sklearn.metrics import accuracy_score        # 匯入準確度計算工具
from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.metrics import accuracy_score           # 匯入準確度計算工具

# 模型訓練

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
   
    
    model = LogisticRegression(random_state=1012, solver='liblinear')     # 創造LogisticRegression模型
    model.fit(train_x_split, train_y_split.values.ravel())  # 訓練LogisticRegression模型
    
    train_pred_y = model.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)
    

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))

#計算整體整確度
model = LogisticRegression(random_state=512, solver='liblinear')
model.fit(train_x, train_y)           
    
train_pred_y = model.predict(train_x)       
train_acc = accuracy_score(train_y, train_pred_y)
print(f'Total accuracy: {train_acc}')


average train accuracy: 0.5477030850224833
    min train accuracy: 0.5422418553285477
    max train accuracy: 0.5538376587520707
average valid accuracy: 0.5477016546523668
    min valid accuracy: 0.5231788079470199
    max valid accuracy: 0.5695364238410596
Total accuracy: 0.5477031802120141


c:\python38\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## RandomForestClassifer

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer          # 匯入填補缺失值的工具
from sklearn.preprocessing import LabelEncoder    # 匯入 Label Encoder
from sklearn.metrics import accuracy_score        # 匯入準確度計算工具
from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.metrics import accuracy_score            # 匯入準確度計算工具

train_x = df[['Open Price','Low Price','High Price']]
train_y = df[['Move']]

# RandomForestClassifier

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
    
    model = RandomForestClassifier(n_estimators = 40, random_state=1012) # 創造RandomForestClassifier模型
    model.fit(train_x_split, train_y_split)           # 訓練RandomForestClassifier模型
    
    train_pred_y = model.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))

#計算整體整確度
model = RandomForestClassifier(n_estimators = 40, random_state=1012) 
model.fit(train_x, train_y)           
    
train_pred_y = model.predict(train_x)       
train_acc = accuracy_score(train_y, train_pred_y)
print(f'Total accuracy: {train_acc}')

<ipython-input-11-e323801b7c2e>:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x_split, train_y_split)           # 訓練RandomForestClassifier模型
<ipython-input-11-e323801b7c2e>:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x_split, train_y_split)           # 訓練RandomForestClassifier模型
<ipython-input-11-e323801b7c2e>:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x_split, train_y_split)           # 訓練RandomForestClassifier模型
<ipython-input-11-e323801b7c2e>:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ra

average train accuracy: 0.998785445334679
    min train accuracy: 0.9977924944812362
    max train accuracy: 0.9994478188845941
average valid accuracy: 0.6859315477934712
    min valid accuracy: 0.6349557522123894
    max valid accuracy: 0.7240618101545254


<ipython-input-11-e323801b7c2e>:57: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x, train_y)


Total accuracy: 0.9995583038869258


In [ ]:
# 請勿更動此區塊程式碼

EXECUTION_END_TIME = time.time() # 計算執行時間
print('total execution time: {}'.format(EXECUTION_END_TIME - EXECUTION_START_TIME))